<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/beegfs/webster/fig3_dfly' 

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix = '#3-pv_fov55' #1
name_prefix2 = '#4-pv_fov55' #2

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>

### Warp using lectin deformation fields 
[13. Flatten warp](#flattenwarp)<br>
[14. TPS lectin warp](#anchorwarp)<br>

### Warp using PV and NFH 
[Using PV/GFAP/Lectin](#pvgfapwarp)<br>
[Using NFH](#nfh)<br>

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

In [ ]:
# Parameters for converting to zarr

slab_path = '/mnt/beegfs/juhyuk/Dfly/2021-12-24-#3R1.po.lec.gfap.nfh.pv/Channel2_tiffs'
slab_zarr_path = bdir(name_prefix+'.zarr')
pc2_img_size = (31505,18748,414)


## Optional parameters 
load_num_slices = 20 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is 0.277 x 0.277 x 1
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
xrange = [7800,11800] # default None
yrange = [8000,12000] # default None
zrange = [14,414] # default None

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:7800-11800, y:8000-12000, z:14-34


  0%|          | 0/20 [00:00<?, ?it/s]TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored

TiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored



TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored



TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored


TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored


TiffFile: the 'fastij' argument is ignored
100%|██████████| 20/20 [01:17<00:00,  3.87s/it]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 400/400 [00:18<00:00, 21.65it/s]

Processing chunk x:7800-11800, y:8000-12000, z:34-54


...


  0%|          | 0/20 [00:00<?, ?it/s]TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored

TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored

TiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignored

TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored


TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored
TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored


TiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignoredTiffFile: the 'fastij' argument is ignored


100%|██████████| 20/20 [01:29<00:00,  4.45s/it]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 400/400 [00:34<00:00, 11.67it/s]

Processing chunk x:7800-11800, y:8000-12000, z:94-114



  0%|          | 0/20 [00:00<?, ?it/s]TiffFile: the 'fastij' argument is ignored

In [12]:
# Convert zarr to tiff
tiff_path = bdir(name_prefix+'_tiffs')
convert_zarr_to_tiff(bdir(name_prefix+'.zarr'), tiff_path, num_workers=None)

Loading z 0 - 200


100%|██████████| 200/200 [06:19<00:00,  1.37s/it]


Loading z 200 - 400


100%|██████████| 200/200 [06:24<00:00,  1.38s/it]


## Convert to zarr of the bottom slab

In [6]:
# Convert from already cropped tiff file (unstitched)
slab_path = bdir("#4-pv_fov55.tif")
slab_zarr_path = bdir('#4-pv_fov55.zarr')
pc2_img_size = (2048,2048,315)


## Optional parameters 
resample_factor = (1,1) # original is 0.277 x 0.277 x 1
chunks = (200,)*3

## crop
xrange = [0,2048] # default None
yrange = [0,2048] # default None
zrange = [0,315] # default None

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, chunks=chunks,
                         lateral_rotate_angle=lateral_rotate_angle, flip=flip, 
                        crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

TiffFile: the 'fastij' argument is ignored


471.0419566631317 seconds elapsed


# Surface flattening

[Return to top](#top)

<a id='flatten'></a>

## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

In [3]:
# Bottom slab 
moving_pts_paths = [bdir('#3-lec_anchor_pts_flat.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('#3-lec_anchor_pts_flat.npy')]

fixed_zarr_path = bdir(name_prefix+'.zarr') 
moving_zarr_path = bdir(name_prefix+'.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(32,)
chunks=3*(200,)
nb_workers = 8 

# affine parameters 
R_path = None
b_path = None

# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir("grid_#3_flatten_fov55.npy")

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None

##########################

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

Loading grid values...
Warping image...


100%|██████████| 800/800 [02:25<00:00,  5.37it/s]


Time elapsed: 9.126044 minutes
Loading z 0 - 200


100%|██████████| 200/200 [03:19<00:00,  1.15s/it]


Loading z 200 - 400


100%|██████████| 200/200 [03:12<00:00,  1.22s/it]


In [4]:
# Bottom slab 
moving_pts_paths = [bdir('#3-lec_anchor_pts_flat.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('#3-lec_anchor_pts_flat.npy')]

fixed_zarr_path = bdir(name_prefix2+'.zarr') 
moving_zarr_path = bdir(name_prefix2+'.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(32,)
chunks=3*(200,)
nb_workers = 8 

# affine parameters 
R_path = None
b_path = None

# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir("grid_#4_flatten_fov55.npy")

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None

##########################

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

Loading grid values...
Warping image...


100%|██████████| 242/242 [00:39<00:00, 12.34it/s]


Time elapsed: 2.462646 minutes
Loading z 0 - 200


100%|██████████| 200/200 [01:06<00:00,  2.35it/s]


Loading z 200 - 315


100%|██████████| 115/115 [00:31<00:00,  3.79it/s]


## TPS warp using lectin grid

[Return to top](#top)

<a id='anchorwarp'></a>

In [5]:
moving_pts_paths = [bdir('#3-lec_anchor_pts_flat.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('#3-lec_anchor_pts_flat.npy')]

fixed_zarr_path = bdir(name_prefix2+'_flattened.zarr') 
moving_zarr_path = bdir(name_prefix2+'_flattened.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_lectps.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(32,)
chunks=3*(200,)
nb_workers = 8 

# affine parameters 
R_path = bdir('R.npy')
b_path = bdir('b.npy')

# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir("grid_anchor_tps_allanchors_r2_fov55.npy")

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None

##########################

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

Loading grid values...
Warping image...


100%|██████████| 363/363 [00:41<00:00,  8.71it/s]


Time elapsed: 1.607941 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:48<00:00,  4.39it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:55<00:00,  3.62it/s]


Loading z 400 - 530


100%|██████████| 130/130 [00:29<00:00,  4.21it/s]


# PV GFAP warp

In [3]:
moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

fixed_zarr_path = bdir(name_prefix2+'_flattened_lectps.zarr') 
moving_zarr_path = bdir(name_prefix2+'_flattened_lectps.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_lectps_pvgfaptps.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8 

# affine parameters 
R_path = bdir('R_pvgfap_fov55.npy')
b_path = bdir('b_pvgfap_fov55.npy')

# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('grid_pvgfap_tps_fov55.npy')

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None

##########################

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

Loading grid values...
Warping image...


100%|██████████| 363/363 [00:35<00:00, 10.11it/s]


Time elapsed: 1.388800 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:37<00:00,  4.51it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:38<00:00,  5.90it/s]


Loading z 400 - 526


100%|██████████| 126/126 [00:23<00:00,  5.51it/s]


In [4]:
moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

fixed_zarr_path = bdir(name_prefix2+'_flattened_lectps_pvgfaptps.zarr') 
moving_zarr_path = bdir(name_prefix2+'_flattened_lectps_pvgfaptps.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_lectps_pvgfaptps_nfhtps.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8 

# affine parameters 
R_path = bdir('R_nfh_fov55.npy')
b_path = bdir('b_nfh_fov55.npy')

# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('grid_pvgfap_nfh_tps_fov55.npy')

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None

##########################

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

Loading grid values...
Warping image...


100%|██████████| 363/363 [00:36<00:00,  9.84it/s]


Time elapsed: 0.975087 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:37<00:00,  5.39it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:48<00:00,  5.21it/s]


Loading z 400 - 532


100%|██████████| 132/132 [00:34<00:00,  4.74it/s]
